# 라이브러리 추가

In [6]:
import pandas as pd
import os
import sys
import json
from collections import Counter

# 파일 읽어오기   
* 파일 경로를 가져온다.
    * 만약 google drive를 통해 파일을 가져온다면 아래와 같이 파일 경로를 찾아 오자.
    * `!find /content/drive/MyDrive/ -name read/*`

In [ ]:
file_path = input("file path plz :)(ex) /content/drive/MyDrive/kakao arena/data/read)>>")

# .2019010120_2019010121.un~ 삭제

# read 디렉터리 내의 파일 리스트를 가지고 옵니다.
read_file_list = os.listdir(file_path)  
read_list = []  # file_name : 파일명
                # user_id : user id
                # content_id : 읽은 글 id

for file_name in read_file_list:
    tmp_path = os.path.join(file_path, file_name)
    with open(tmp_path, 'r') as f:
        for line in f:
            line_list = line.split()
            read_list.append({"file_name":file_name,"user_id":line_list[0],"content_id":line_list[1:]})

read_list = pd.DataFrame(read_list)

read_list.to_pickle(file_path + ".pkl")

read_list.head(5)

,file_name,user_id,content_id
0,2019012301_2019012302,#6b77f7aadb7ea35aa91ec34987ac0b5a,[@pinastro_6]
1,2019012301_2019012302,#fdde4774182dfe1f8f87aa976d94bab0,"[@dailylife_666, @munchi_78, @haereka_93, @sun..."
2,2019012301_2019012302,#859958bb4e89736c4a98fc861e15d691,"[@brunchtq1f_42, @iswim_12, @nrlee0610_16]"
3,2019012301_2019012302,#1f6c9f5a0e4dc284a142212bce672940,"[@ggoforit_46, @tenbody_1631, @sweetannie_140,..."
4,2019012301_2019012302,#afb958144010a82523a266f413432e80,"[@bookerbuker_267, @bookerbuker_267, @bookerbu..."


* read 디렉토리: 본 글 정보

  * read.tar은 2018년 10월 1일부터 2019년 3월 1일까지 일부 브런치 독자들이 본 글의 정보가 총 3,625개의 파일로 구성되어 있습니다.
  * 파일의 이름은 시작일_종료일 형태입니다. 예를 들어 2018110708_2018110709 파일은 2018년 11월 7일 오전 8시부터 2018년 11월 7일 오전 9시 전까지 본 글입니다.
  * 파일은 여러 줄로 이뤄져 있으며 하나의 줄은 브런치의 독자가 파일의 시간 동안 본 글을 시간 순으로 기록한 것입니다. 한 줄의 정보는 공백으로 구분되어 있으며 첫번 째가 독자의 암호화된 식별자이고 그 뒤로는 해당 독자가 본 글의 정보입니다.
  * 예를 들어 read/2019022823_2019030100 파일에 기록된 아래 정보는 #8a706ac921a11004bab941d22323efab 라는 독자가 2019년 2월 28일 23시에서 2019년 3월 1일 0시 사이에 @bakchacruz_34 @wo-motivator_133 @wo-motivator_133 를 순서대로 보았다는 뜻입니다. @wo-motivator_133 글이 두 번 나타난 것은 이 글을 보기 위해 두 번 방문했다는 뜻입니다.
    * #8a706ac921a11004bab941d22323efab @bakchacruz_34 @wo-motivator_133 @wo-motivator_133
  * 글을 보았다는 의미는 특정 글에 모바일, PC, 앱을 통해 접근했다는 뜻입니다. 머문 시간에 대한 정보가 제공되지 않기 때문에 실제로 글을 읽지 않고 이탈하는 등의 가능성도 있습니다.

# 조회수

In [7]:
content = pd.DataFrame(Counter([j for i in read_list["content_id"].tolist() for j in i]).most_common())

content.head(5)

,0,1
0,@brunch_141,97206
1,@brunch_151,66903
2,@brunch_145,31162
3,@tenbody_1305,28922
4,@intlovesong_28,23760
